In [1]:
import numpy as np
import pandas as pd

# acquire
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import os

# prepare
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# explore
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# model
from sklearn.linear_model import LogisticRegression
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report



In [2]:
import acquire
import prepare
import wrangle
import github_acquire

credentials loaded successfully
Wrangle functions loaded successfully.
git acquire module loaded successsfully


In [3]:
# df_raw = pd.read_json('repos.json')
# df.head()

In [4]:
# len(df_raw.language[df_raw.language.isnull()])

In [5]:
df = pd.read_json('repos_clean.json')
df.head()

,language,repo,content,stemmed,lemmatized,clean
0,JavaScript,facebook/react,React · \nReact is a JavaScript library for...,react react is a javascript librari for build ...,react react is a javascript library for buildi...,react react javascript library building user i...
1,JavaScript,d3/d3,D3: Data-Driven Documents\n\nD3 (or D3.js) is ...,d3 datadriven document d3 or d3j is a javascri...,d3 datadriven document d3 or d3js is a javascr...,d3 datadriven document d3 d3js javascript libr...
2,JavaScript,vuejs/vue,\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue.j...,support vuej vuej is an mitlicens open sourc p...,supporting vuejs vuejs is an mitlicensed open ...,supporting vuejs vuejs mitlicensed open source...
3,JavaScript,axios/axios,axios\n\n\n\n\n\n\n\n\nPromise based HTTP clie...,axio promis base http client for the browser a...,axios promise based http client for the browse...,axios promise based http client browser nodejs...
4,JavaScript,facebook/create-react-app,Create React App \n\nCreate React apps with n...,creat react app creat react app with no build ...,create react app create react apps with no bui...,create react app create react apps build confi...


In [6]:
df.sample(25)

,language,repo,content,stemmed,lemmatized,clean
344,JavaScript,jhipster/generator-jhipster,"\n \n \nGreetings, Java Hipster!\nFull do...",greet java hipster full document and inform is...,greeting java hipster full documentation and i...,greeting java hipster full documentation infor...
157,Python,TheAlgorithms/Python,The Algorithms - Python\n \n \n \n \n \n \n \n...,the algorithm python all algorithm implement i...,the algorithm python all algorithm implemented...,algorithm python algorithm implemented python ...
281,Python,miloyip/game-programmer,\nEnglish svg pdf jpg png\n简体中文 svg pdf jpg pn...,english svg pdf jpg png svg pdf jpg png by tkc...,english svg pdf jpg png svg pdf jpg png by tkc...,english svg pdf jpg png svg pdf jpg png tkchu ...
259,Python,ray-project/ray,"\n\n\n\n\n\nRay provides a simple, universal A...",ray provid a simpl univers api for build distr...,ray provides a simple universal api for buildi...,ray provides simple universal api building dis...
347,JavaScript,Popmotion/popmotion,\nThe animator's toolbox\n\n\n\nPopmotion is:\...,the anim toolbox popmot is power it support ke...,the animator toolbox popmotion is powerful it ...,animator toolbox popmotion powerful support ke...
399,JavaScript,statsd/statsd,StatsD \nA network daemon that runs on the N...,statsd a network daemon that run on the nodej ...,statsd a network daemon that run on the nodejs...,statsd network daemon run nodejs platform list...
11,JavaScript,angular/angular.js,AngularJS \nAngularJS lets you write client-si...,angularj angularj let you write clientsid web ...,angularjs angularjs let you write clientside w...,angularjs angularjs let write clientside web a...
327,JavaScript,segmentio/nightmare,\n\nNightmare\nNightmare is a high-level brows...,nightmar nightmar is a highlevel browser autom...,nightmare nightmare is a highlevel browser aut...,nightmare nightmare highlevel browser automati...
532,Python,deepinsight/insightface,InsightFace: 2D and 3D Face Analysis Project\n...,insightfac 2d and 3d face analysi project by j...,insightface 2d and 3d face analysis project by...,insightface 2d 3d face analysis project jia gu...
226,Python,google/python-fire,Python Fire \nPython Fire is a library for aut...,python fire python fire is a librari for autom...,python fire python fire is a library for autom...,python fire python fire library automatically ...


In [ ]:
# # add a column that is a list of each word for each repo --> PREPARE.PY
# words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.clean] 

# # column name will be words, and the column will contain lists of the words in each doc
# df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)
# df.head()

### Splitting the data

In [ ]:


from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df[['language', 'lemmatized', 'clean']], 
                                        stratify=df.language, 
                                        test_size=.2, random_state = 123)

train, validate = train_test_split(train_validate, 
                                   stratify=train_validate.language, 
                                   test_size=.25, random_state = 123)


X_train = train.drop(columns = [target_variable])

In [9]:
y_train = train['language']
y_train

277        Python
97     JavaScript
519        Python
203        Python
582        Python
          ...    
307    JavaScript
527        Python
552        Python
249        Python
264        Python
Name: language, Length: 348, dtype: object

In [ ]:
# what's the proportion of each language in our data
labels = pd.concat([train.language.value_counts(), # get total counts of each language
                    train.language.value_counts(normalize=True)], axis=1) # getting the prop of each language
labels.columns = ['n', 'percent']
labels

In [7]:
# split the data using train_test_split, doing it twice so that we have 
from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df[['language', 'lemmatized', 'clean']], 
                                        stratify=df.language, 
                                        test_size=.2, random_state = 123)

train, validate = train_test_split(train_validate, 
                                   stratify=train_validate.language, 
                                   test_size=.25, random_state = 123)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train_labels = pd.concat([train.language.value_counts(), # get total counts of ham vs spam
                    train.language.value_counts(normalize=True)], axis=1) # getting the prop of ham vs. spam

train_labels.columns = ['n', 'percent']
train_labels

#### Bag of Words Setup

In [ ]:
# Create CountVectorizer, which create bag-of-words model.
# stop_words : Specify language to remove stopwords. 
# min_df: ignore terms that have a document frequency strictly 
# lower than the given threshold. This value is also called cut-off in the literature. 
# If float, the parameter represents a proportion of documents, integer absolute counts. 
# ngram_range: the lower and upper boundary of the range of n-values for 
# different word n-grams or char n-grams to be extracted. 

vectorizer = CountVectorizer(stop_words='english', 
                             min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

# Learn vocabulary in sentences. 
vectorizer.fit(train.clean)

# Get dictionary. 
vectorizer.get_feature_names()

In [ ]:
# Transform each sentences in vector space.
bow = vectorizer.transform(train.clean)

# this is just to see the array of 0's and 1's
bow_array = bow.toarray()
bow_array[0]

#### TD-IDF Setup

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

tfidf_sparse_matrix = tfidf.fit_transform(train.clean)
tfidf_sparse_matrix

In [ ]:
pd.DataFrame(tfidf_sparse_matrix.todense(), columns=tfidf.get_feature_names()).head()

In [ ]:
# Get vocabularies.
tfidf.vocabulary_

In [ ]:
# Transform to document-term matrix
vector_spaces = tfidf.transform(X_train.clean)
vector_spaces.toarray()

#### Creating the X_train and y_train variables for modeling:

In [ ]:
# Assigning the target:
y = train['language']

# Assigning the Features:
X_bow = bow

X_tfidf = tfidf_sparse_matrix

In [ ]:
# Predicting based on BoW:

from sklearn.linear_model import LogisticRegression

lm = LogisticRegression().fit(X_bow, y)

train['predicted'] = lm.predict(X_bow)
train.head()

In [ ]:
confusion_matrix(train.language, train.predicted)
pd.crosstab(train.language, train.predicted)

In [ ]:
print(classification_report(train.language, train.predicted))

In [ ]:
lm_tfidf = LogisticRegression().fit(X_tfidf, y)
train['pred_tfidf'] = lm_tfidf.predict(X_tfidf)

In [ ]:
print(classification_report(train.language, train.pred_tfidf))

In [ ]:
validate.shape